# Start

In [39]:
from kinetic_solver_v3 import *
import argparse 
import time

# Input
parser = argparse.ArgumentParser(
    description='Perform kinetic modelling given the free energy profile and mechanism detail')

parser.add_argument(
    "-i",
    help="input reaction profiles in csv"
    )

parser.add_argument("-a", 
                    help="manually add an input reaction profile in csv", 
                    action="append")

parser.add_argument(
    "-c",
    "--c",
    type=str,
    default="c0.txt",
    help="text file containing initial concentration of all species [[INTn], [Rn], [Pn]]")

parser.add_argument(
    "-r",
    "--r",
    type=str,
    default="Rp.txt",
    help="reactant position matrix")

parser.add_argument(
    "-p",
    "--p",
    type=str,
    default="Pp.txt",
    help="product position matrix")

parser.add_argument(
    "-rn",
    "--rn",
    type=str,
    default="rxn_network,csv",
    help="reaction network matrix")

parser.add_argument(
    "--Time",
    type=float,
    default=1e8,
    help="total reaction time (s)")

parser.add_argument(
    "-t",
    "--t",
    type=float,
    default=298.15,
    help="temperature (K)")

parser.add_argument(
    "-de",
    "--de",
    type=str,
    default="LSODA",
    help="Integration method to use (odesolver)")

_StoreAction(option_strings=['-de', '--de'], dest='de', nargs=None, const=None, default='LSODA', type=<class 'str'>, choices=None, required=False, help='Integration method to use (odesolver)', metavar=None)

In [40]:
dir = "test_cases/1/"

args = parser.parse_args(['-i', f"{dir}/reaction_data", 
                          '-c', f"{dir}/c0.txt",
                          "-r", f"{dir}/Rp.txt",
                          "-rn", f"{dir}/rxn_network.csv",
                          "-p", f"{dir}/Pp.txt",
                          ])
initial_conc, Rp, Pp, t_span, temperature, method, energy_profile_all, dgr_all, coeff_TS_all, rxn_network = load_data(args)
k_forward_all, k_reverse_all, Rp_, Pp_, n_INT_all = process_data(Rp, Pp, energy_profile_all, dgr_all, coeff_TS_all, rxn_network, temperature)

FileNotFoundError: [Errno 2] No such file or directory: 'test_cases/1//reaction_data'

In [ ]:
dir = "test_cases/1/"

args = parser.parse_args(['-i', f"{dir}/reaction_data", 
                          '-c', f"{dir}/c0.txt",
                          "-r", f"{dir}/Rp.txt",
                          "-rn", f"{dir}/rxn_network.csv",
                          "-p", f"{dir}/Pp.txt",
                          ])

initial_conc, Rp, Pp, t_span, temperature, method, energy_profile_all, dgr_all, coeff_TS_all, rxn_network = load_data(args)
k_forward_all, k_reverse_all, Rp_, Pp_, n_INT_all = process_data(Rp, Pp, energy_profile_all, dgr_all, coeff_TS_all, rxn_network, temperature)

dydt = system_KE(
    k_forward_all,
    k_reverse_all,
    rxn_network,
    Rp_,
    Pp_,
    n_INT_all,
    initial_conc,
    jac_method="ag")

t_eval = np.linspace(t_span[0], t_span[1], 250)

start_time = time.time()
result_solve_ivp = solve_ivp(
    dydt,
    t_span,
    initial_conc,
    method="BDF",
    dense_output=True,
    t_eval=t_eval,
    # first_step=first_step,
    # max_step=max_step,
    rtol=1e-3,
    atol=1e-6,
    jac=dydt.jac,
)
plot_save(result_solve_ivp, rxn_network, Rp_, Pp_, dir)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

# test diffrax

In [ ]:
import time
import diffrax
import equinox as eqx  # https://github.com/patrick-kidger/equinox
import jax
import jax.numpy as jnp

jax.config.update("jax_enable_x64", True)

k_forward = k_forward_all[0]
k_reverse = k_reverse_all[0]

class system_KE_dfx(eqx.Module):
    
    k_forward: jax.numpy.ndarray
    k_reverse: jax.numpy.ndarray

    def __call__(self, t, y, args):
        
        k = 7
        dydt = [None for _ in range(k)] 
        dydt[0] = -self.k_forward[0]*y[0]*y[4] + self.k_reverse[0]*y[1] + self.k_forward[3]*y[3] - self.k_reverse[3]*y[0]*y[6]
        dydt[1] = self.k_forward[0]*y[0]*y[4] - self.k_reverse[0]*y[1] - self.k_forward[1]*y[1]*y[5] + self.k_reverse[1]*y[2]
        dydt[2] = self.k_forward[1]*y[1]*y[5] -  self.k_reverse[1]*y[2] - self.k_forward[2]*y[2] + self.k_reverse[2]*y[3]
        dydt[3] = self.k_forward[2]*y[2] - self.k_reverse[2]*y[3] - self.k_forward[3]*y[3] + self.k_reverse[3]*y[0]*y[6]
        dydt[4] = -self.k_forward[0]*y[0]*y[4] + self.k_reverse[0]*y[1]
        dydt[5] = -self.k_forward[1]*y[1]*y[5] + self.k_reverse[1]*y[2]
        dydt[6] = self.k_forward[3]*y[3] - self.k_reverse[3]*y[0]*y[6]
        
        return jnp.array(dydt)

time_span = (0, 1e8)
t_eval = jnp.linspace(0, 1e8, 200)

@jax.jit
def main(k_forward, k_reverse):
    km = system_KE_dfx(k_forward, k_reverse)
    terms = diffrax.ODETerm(km)
    
    t0 = time_span[0]
    t1 = time_span[1]
    y0 = jnp.array(initial_conc)
    dt0 = 0.0002
    solver = diffrax.Kvaerno5()
    saveat = diffrax.SaveAt(ts=t_eval)
    stepsize_controller = diffrax.PIDController(rtol=1e-3, atol=1e-6)
    sol = diffrax.diffeqsolve(
        terms,
        solver,
        t0,
        t1,
        dt0,
        y0,
        saveat=saveat,
        stepsize_controller=stepsize_controller,
    )
    return sol


start = time.time()
sol = main(k_forward, k_reverse)
end = time.time()

# print("Results:")
# for ti, yi in zip(sol.ts, sol.ys):
#     print(f"t={ti.item()}, y={yi.tolist()}")
print(f"Took {sol.stats['num_steps']} steps in {end - start} seconds.")

plt.plot(np.log10(sol.ts), sol.ys[:, 0], '-b', linewidth=2, label='cat')
plt.plot(np.log10(sol.ts), sol.ys[:, 4], '--g', linewidth=2, label='R1')
plt.plot(np.log10(sol.ts), sol.ys[:, 5], '--', color="cyan", linewidth=2, label='R2')
plt.plot(np.log10(sol.ts), sol.ys[:, 6], '-r', linewidth=2, label='product')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('log(time, s)')
plt.ylabel('Concentration (mol/l)')
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

# revising the input for the reaction network


#### to combine Rp, Pp, and reaction_network into one single inp

- from there, in the process_data function, form Rp, Pp, and reaction_network

- have user specify number of R and P

In [ ]:
import numpy as np
import pandas as pd

def pad_network(X, n_INT_all, rxn_network):

    X_ = np.array_split(X, np.cumsum(n_INT_all)[:-1])
    # the first profile is assumed to be full, skipped
    insert_idx = []
    for i in range(1, len(n_INT_all)):  # n_profile - 1
        # pitfall
        if np.all(rxn_network[np.cumsum(n_INT_all)[i - 1]:np.cumsum(n_INT_all)[i], 0] == 0):
            cp_idx = np.where(rxn_network[np.cumsum(n_INT_all)[
                                i - 1]:np.cumsum(n_INT_all)[i], :][0] == -1)
            tmp_idx = cp_idx[0][0].copy()
            all_idx = [tmp_idx]
            while tmp_idx != 0:
                tmp_idx = np.where((rxn_network[tmp_idx, :] == -1))[0][0]
                all_idx.insert(0, tmp_idx)
            X_[i] = np.insert(X_[i], 0, X[all_idx], axis=0)
            insert_idx.append(all_idx)

        else:
            all_idx = []
            for j in range(rxn_network.shape[0]):
                if j >= np.cumsum(n_INT_all)[
                        i - 1] and j <= np.cumsum(n_INT_all)[i]:
                    continue
                elif np.any(rxn_network[np.cumsum(n_INT_all)[i - 1]:\
                    np.cumsum(n_INT_all)[i], j]):
                    X_[i] = np.insert(X_[i], j, X[j], axis=0)
                    all_idx.append(j)
            insert_idx.append(all_idx)
                    
    return X_, insert_idx

### all reaction data in one csv

- [INT TS]_1 P_1 [INT TS]_2 P_2 ....
- in case of the pitfall, treat it as P: P-INT

2A as example

In [ ]:
df_network = pd.read_csv("test_cases/2A_3/rxn_network.csv")

# process reaction network
rxn_network_all = df_network.to_numpy()[:, 1:]
rxn_network_all = rxn_network_all.astype(np.int64)
states = df_network.columns[1:].tolist()
nR = len([s for s in states if s.lower().startswith('r') and 'INT' not in s])
nP = len([s for s in states if s.lower().startswith('p') and 'INT' not in s])

n_INT_tot = 0
for i in range(rxn_network_all.shape[0]):
    try:
        if rxn_network_all[i+1,i] == -1: n_INT_tot+=1
        elif rxn_network_all[0,i] ==-1 and rxn_network_all[i+2,i+1] == -1:
            n_INT_tot+=1
        elif rxn_network_all[0,i] ==-1 and rxn_network_all[i+2,i+1] == 0:
            n_INT_tot+=1
            break
    except IndexError as e:
        break
    
rxn_network = rxn_network_all[:n_INT_tot, :n_INT_tot]

n_INT_all = []
x = 1
for i in range(1, rxn_network.shape[0]):
    if rxn_network[i, i - 1] == -1:
        x += 1
    elif rxn_network[i, i - 1] != -1:
        n_INT_all.append(x)
        x = 1
n_INT_all.append(x)
n_INT_all = np.array(n_INT_all)

Rp, _ = pad_network(rxn_network_all[:n_INT_tot, n_INT_tot:n_INT_tot+nR], n_INT_all, rxn_network)
Pp, idx_insert = pad_network(rxn_network_all[:n_INT_tot, n_INT_tot+nR:], n_INT_all, rxn_network)


# single csv to seperate csv for each profile
df_all = pd.read_csv("test_cases/2A_3/reaction_data.csv")
species_profile = df_all.columns.values[1:]

all_df = []
df_ = pd.DataFrame({'R': np.zeros(len(df_all))})
for i in range(1,len(species_profile)):
    if species_profile[i].lower().startswith("p"): 
        df_ = pd.concat([df_, df_all[species_profile[i]]], ignore_index=False, axis=1)
        all_df.append(df_)
        df_ = pd.DataFrame({'R': np.zeros(len(df_all))})
    else: df_ = pd.concat([df_, df_all[species_profile[i]]], ignore_index=False, axis=1)
    
for idx in idx_insert:
    all_state_insert = [states[i] for i in idx]
    for j, state in list(enumerate(species_profile[::-1])):
        if state in all_state_insert and j != len(species_profile) - 1: 
            all_df[idx[0] + 1].insert(1, state, df_all[state].values)
        elif "TS" in state and species_profile[::-1][j - 1] in all_state_insert: 
            all_df[idx[0] + 1].insert(1, state, df_all[state].values)

energy_profile_all = []
dgr_all = []
coeff_TS_all = []
for df in all_df:
    energy_profile = df.values[0][0:-1]
    rxn_species = df.columns.to_list()[1:-1]
    dgr_all.append(df.values[0][-1])
    coeff_TS = [1 if "TS" in element else 0 for element in rxn_species]
    coeff_TS_all.append(np.array(coeff_TS))
    energy_profile_all.append(np.array(energy_profile))

## simplify c0

- manually specify everything: detect by len of c0
- if len(c0) != 

In [ ]:
c0 = np.array([0.2, 0.8, 0.9])

df_network = pd.read_csv("test_cases/2A_3/rxn_network.csv")
rxn_network_all = df_network.to_numpy()[:, 1:]
rxn_network_all = rxn_network_all.astype(np.int64)
states = df_network.columns[1:].tolist()
nR = len([s for s in states if s.lower().startswith('r') and 'INT' not in s])
nP = len([s for s in states if s.lower().startswith('p') and 'INT' not in s])
n_INT_tot = rxn_network_all.shape[0] - nR - nP

if len(c0) != rxn_network_all.shape[1]:
    tmp = np.zeros(rxn_network_all.shape[1])
    for i, c in enumerate(c0):
        if i == 0: tmp[0] = c0[0]
        else: tmp[n_INT_tot + i -1] = c
    
tmp

## simplify rxn network inp

In [ ]:
df_network = pd.read_csv("test_cases/2A_3/rxn_network_simplified.csv", index_col=0)
df_network.fillna(0, inplace=True)
df_network

In [ ]:
names = df[df.columns[0]].values
cb, ms = group_data_points(0, 2, names)
cb

array(['g', 'g', 'g', 'g', 'g', 'g', 'g', 'c', 'c', 'c', 'c', 'c', 'c',
       'c', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'm', 'm', 'm', 'm', 'm',
       'm', 'm', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'g', 'c', 'm', 'm'],
      dtype='<U1')

# Combining with volcanic

### read files and gen SPRs

In [ ]:
from navicat_volcanic.helpers import (arraydump, group_data_points,
                                      processargs, setflags, user_choose_1_dv,
                                      user_choose_2_dv, bround)
from navicat_volcanic.plotting2d import get_reg_targets, plot_2d
from navicat_volcanic.dv1 import curate_d, find_1_dv
from navicat_volcanic.tof import calc_tof
from navicat_volcanic.helpers import group_data_points
import scipy.stats as stats
import pandas as pd
import numpy as np
import time
from kinetic_solver_v3 import *
from tqdm import tqdm

# user inp
dir = "volcanic_test/formate/"
temperature = 298
t_span = (0, 1e5)


filename = f"{dir}reaction_data.xlsx"
c0 = f"{dir}c0.txt"
df_network = pd.read_csv(f"{dir}rxn_network.csv")
verb = 0
T = 298.15
interpolate = True
n_points = 100
threshold_diff = 0.1
timeout = 15 # in seconds

df = pd.read_excel(filename)
names = df[df.columns[0]].values
cb, ms = group_data_points(0, 2, names)
tags = np.array([str(tag) for tag in df.columns[1:]], dtype=object)
d = np.float32(df.to_numpy()[:, 1:])

coeff = np.zeros(len(tags), dtype=bool)
regress = np.zeros(len(tags), dtype=bool)
for i, tag in enumerate(tags):
    if "TS" in tag.upper():
        if verb > 0:
            print(f"Assuming field {tag} corresponds to a TS.")
        coeff[i] = True
        regress[i] = True
    elif "DESCRIPTOR" in tag.upper():
        if verb > 0:
            print(
                f"Assuming field {tag} corresponds to a non-energy descriptor variable."
            )
        start_des = tag.upper().find("DESCRIPTOR")
        tags[i] = "".join(
            [i for i in tag[:start_des]] + [i for i in tag[start_des + 10 :]]
        )
        coeff[i] = False
        regress[i] = False
    elif "PRODUCT" in tag.upper():
        if verb > 0:
            print(f"Assuming ΔG of the reaction(s) are given in field {tag}.")
        dgr = d[:, i]
        coeff[i] = False
        regress[i] = True
    else:
        if verb > 0:
            print(f"Assuming field {tag} corresponds to a non-TS stationary point.")
        coeff[i] = False
        regress[i] = True
        
lmargin=35
rmargin=35
npoints=250
plotmode=1
verb=0

npoints=250
xbase = 20
ybase = 5

dvs, r2s = find_1_dv(d, tags, coeff, regress, verb)
idx = user_choose_1_dv(dvs, r2s, tags) # choosing descp


X, tag, tags, d, d2, coeff = get_reg_targets(idx, d, tags, coeff, regress, mode="k")
descp_idx = np.where(tag == tags)[0][0]
lnsteps = range(d.shape[1])
xmax = bround(X.max() + rmargin, xbase)
xmin = bround(X.min() - lmargin, xbase)
if verb > 1:
    print(f"Range of descriptor set to [ {xmin} , {xmax} ]")
xint = np.linspace(xmin, xmax, npoints)
dgs = np.zeros((npoints, len(lnsteps)))
for i, j in enumerate(lnsteps):
    Y = d[:, j].reshape(-1)
    p, cov = np.polyfit(X, Y, 1, cov=True)
    Y_pred = np.polyval(p, X)
    n = Y.size
    m = p.size
    dof = n - m
    t = stats.t.ppf(0.95, dof)
    resid = Y - Y_pred
    with np.errstate(invalid="ignore"):
        chi2 = np.sum((resid / Y_pred) ** 2)
    s_err = np.sqrt(np.sum(resid**2) / dof)
    yint = np.polyval(p, xint) # 
    dgs[:, i] = yint
coeff_TS_all = [coeff[:-1].astype(int)]

Disagreement: best descriptors is either 
INT3 or 
INT4

INT3 has been identified as a suitable descriptor variable with r2=0.9134.


- dgs: SPRs
- yint = reaction energy

### compute for volcano point/line with the activity function (TOF in this case)

In [ ]:

# volcano line
ytof = np.zeros_like(yint)
for i in range(ytof.shape[0]):
    profile = dgs[i, :]
    dgr_s = dgs[i][-1]
    tof = np.log10(calc_tof(profile, dgr_s, T, coeff, exact=True)[0])
    ytof[i] = tof
    
# volcano point   
px = np.zeros_like(d[:, 0])
py = np.zeros_like(d[:, 0])
for i in range(d.shape[0]):
    px[i] = X[i].reshape(-1)
    profile = d[i, :]
    dgr_s = dgr[i]
    tof = np.log10(calc_tof(profile, dgr_s, T, coeff, exact=True)[0])
    py[i] = tof

### Arguements for KM
- rxn_data: dgs
- temperature: initially available on volcanic
- coeff: from volcanic as well
------------------------------------
- initial_conc
- reaction network data
   - Rp
   - Pp
   - rxn_network
- t_span
- method


It is advisable to have inp (rxn_data, network, initial_conc) in certain directory

Assuming single pathway for now

In [112]:
def calc_km(energy_profile_all, dgr_all, temperature, coeff_TS_all, df_network, t_span, c0, timeout):
   
    initial_conc = np.loadtxt(c0, dtype=np.float64)
    df_network.fillna(0, inplace=True)
    rxn_network_all = df_network.to_numpy()[:, 1:]
    rxn_network_all = rxn_network_all.astype(np.int32)
    states = df_network.columns[1:].tolist()
    nR = len([s for s in states if s.lower().startswith('r') and 'INT' not in s])
    nP = len([s for s in states if s.lower().startswith('p') and 'INT' not in s])
    n_INT_tot = rxn_network_all.shape[1] - nR - nP
    rxn_network = rxn_network_all[:n_INT_tot, :n_INT_tot]

    n_INT_all = []
    x = 1
    for i in range(1, rxn_network.shape[1]):
        if rxn_network[i, i - 1] == -1:
            x += 1
        elif rxn_network[i, i - 1] != -1:
            n_INT_all.append(x)
            x = 1
    n_INT_all.append(x)
    n_INT_all = np.array(n_INT_all)

    Rp, _ = pad_network(rxn_network_all[:n_INT_tot, n_INT_tot:n_INT_tot+nR], n_INT_all, rxn_network)
    Pp, _ = pad_network(rxn_network_all[:n_INT_tot, n_INT_tot+nR:], n_INT_all, rxn_network)   

    # pad initial_conc in case [cat, R] are only specified.
    if len(initial_conc) != rxn_network_all.shape[1]:
        tmp = np.zeros(rxn_network_all.shape[1])
        for i, c in enumerate(initial_conc):
            if i == 0: tmp[0] = initial_conc[0]
            else: tmp[n_INT_tot + i -1] = c
        initial_conc = np.array(tmp)
                
    k_forward_all = []
    k_reverse_all = []

    for i in range(len(energy_profile_all)):
        k_forward, k_reverse = get_k(
            energy_profile_all[i], dgr_all[i], coeff_TS_all[i], temperature=temperature)
        k_forward_all.append(k_forward)
        k_reverse_all.append(k_reverse)

    dydt = system_KE(
        k_forward_all,
        k_reverse_all,
        rxn_network,
        Rp,
        Pp,
        n_INT_all,
        initial_conc)
    
    # first try BDF + ag with various rtol and atol
    # then LSODA + FD if all BDF attempts fail
    # the last resort is a Radau
    # if all fail, return NaN
    max_step = (t_span[1] - t_span[0]) / 10.0
    rtol_values = [1e-3, 1e-6, 1e-9, 1e-10 ]
    atol_values = [1e-6, 1e-6, 1e-9, 1e-10]
    last_ = [rtol_values[-1], atol_values[-1]]
    success=False
    cont=False
    while success==False:
        atol = atol_values.pop(0)
        rtol = rtol_values.pop(0)
        try:
            result_solve_ivp = solve_ivp(
                dydt,
                t_span,
                initial_conc,
                method="BDF",
                dense_output=True,
                rtol=rtol,
                atol=atol,
                jac=dydt.jac,
                max_step=max_step,
                timeout=timeout,
            )
            #timeout
            if result_solve_ivp == "Shiki": 
                if rtol == last_[0] and atol == last_[1]:
                    success=True
                    cont=True
                continue
            else: success=True
        
        # should be arraybox failure
        except Exception as e:
            if rtol == last_[0] and atol == last_[1]:
                success=True
                cont=True
            continue

    if cont:
        try:
            result_solve_ivp = solve_ivp(
                dydt,
                t_span,
                initial_conc,
                method="LSODA",
                dense_output=True,
                rtol=1e-6,
                atol=1e-9,
                max_step=max_step,
                timeout=timeout,
            )
        except Exception as e:
            # Last resort
            result_solve_ivp = solve_ivp(
                dydt,
                t_span,
                initial_conc,
                method="Radau",
                dense_output=True,
                rtol=1e-6,
                atol=1e-9,
                max_step=max_step,
                jac=dydt.jac,
                timeout=timeout,
            )
        
    try:     
        idx_target_all = [states.index(i) for i in states if "*" in i]
        c_target_t = [result_solve_ivp.y[i][-1] for i in idx_target_all]
        return c_target_t
    except IndexError as e: 
        return np.NaN

### getting volcano lines

In [ ]:
from tqdm import tqdm

interpolate = True
n_points = 100
threshold_diff = 1
timeout = 20 # in seconds

if interpolate: 
    selected_indices = np.round(np.linspace(0, len(dgs) - 1, n_points)).astype(int)
    trun_dgs = []
    for i in range(len(dgs)):
        if i not in selected_indices: trun_dgs.append([np.nan])
        else: trun_dgs.append(dgs[i])
else: trun_dgs = dgs

prod_conc = []
prev_profile = None
prev_result = None
for profile in tqdm(trun_dgs, total=len(trun_dgs), ncols=80):
    if np.isnan(profile[0]): 
        prod_conc.append(np.nan)
        continue
    else:
        try:
            result = calc_km([profile[:-1]], [profile[-1]], 298.15, coeff_TS_all, df_network, (0, 1e5), c0, timeout=timeout)
            if result[0] is None:
                prod_conc.append(np.nan)
                continue
            if prev_result is None:
                prev_result = result[0]
                prod_conc.append(result[0])
            else:
                diff = abs(result - prev_result)
                if diff < threshold_diff:
                    prev_result = result[0]
                    prod_conc.append(result[0])        
                else:
                    print("diff")
                    prod_conc.append(np.nan)
                    continue
                    
        except Exception as e:
            print(e)
            prod_conc.append(np.nan)

In [ ]:
from scipy.interpolate import interp1d

descr_all = np.array([i[descp_idx] for i in dgs])
prod_conc = np.array([i for i in prod_conc])

#sort both 
sort_indices = np.argsort(descr_all)
descr_all = descr_all[sort_indices]
prod_conc = prod_conc[sort_indices]

# to avoid interpolating beyond the range
if prod_conc[0] == np.NaN:
    descr_all = descr_all[1:]
    prod_conc = prod_conc[1:]
elif prod_conc[-1] == np.NaN:
    descr_all = descr_all[:-1]
    prod_conc = prod_conc[:-1] 

# always choose the first and the last, but what if they fuck up
missing_indices = np.isnan(prod_conc)
f = interp1d(descr_all[~missing_indices], prod_conc[~missing_indices], kind='linear')

y_interp = f(descr_all[missing_indices])

prod_conc_ = prod_conc.copy()
prod_conc_[missing_indices] = y_interp

### volcano points

In [ ]:
profile

array([  0.  ,   1.21, -19.97, -11.1 , -20.89, -10.57, -21.31],
      dtype=float32)

In [ ]:
profile = d[5]
energy_profile_all = [profile[:-1]]
dgr_all = [profile[-1]]
t_span = (0, 1e5)
initial_conc = np.loadtxt(c0, dtype=np.float64)

df_network.fillna(0, inplace=True)
rxn_network_all = df_network.to_numpy()[:, 1:]
rxn_network_all = rxn_network_all.astype(np.int32)
states = df_network.columns[1:].tolist()
nR = len([s for s in states if s.lower().startswith('r') and 'INT' not in s])
nP = len([s for s in states if s.lower().startswith('p') and 'INT' not in s])
n_INT_tot = rxn_network_all.shape[1] - nR - nP
rxn_network = rxn_network_all[:n_INT_tot, :n_INT_tot]

n_INT_all = []
x = 1
for i in range(1, rxn_network.shape[1]):
    if rxn_network[i, i - 1] == -1:
        x += 1
    elif rxn_network[i, i - 1] != -1:
        n_INT_all.append(x)
        x = 1
n_INT_all.append(x)
n_INT_all = np.array(n_INT_all)

Rp, _ = pad_network(rxn_network_all[:n_INT_tot, n_INT_tot:n_INT_tot+nR], n_INT_all, rxn_network)
Pp, _ = pad_network(rxn_network_all[:n_INT_tot, n_INT_tot+nR:], n_INT_all, rxn_network)   

# pad initial_conc in case [cat, R] are only specified.
if len(initial_conc) != rxn_network_all.shape[1]:
    tmp = np.zeros(rxn_network_all.shape[1])
    for i, c in enumerate(initial_conc):
        if i == 0: tmp[0] = initial_conc[0]
        else: tmp[n_INT_tot + i -1] = c
    initial_conc = np.array(tmp)
            
k_forward_all = []
k_reverse_all = []

for i in range(len(energy_profile_all)):
    k_forward, k_reverse = get_k(
        energy_profile_all[i], dgr_all[i], coeff_TS_all[i], temperature=temperature)
    k_forward_all.append(k_forward)
    k_reverse_all.append(k_reverse)

dydt = system_KE(
    k_forward_all,
    k_reverse_all,
    rxn_network,
    Rp,
    Pp,
    n_INT_all,
    initial_conc)

result_solve_ivp = 0
result_solve_ivp.success = False
max_step = (t_span[1] - t_span[0]) / 10.0
rtol_values = [1e-3, 1e-6, 1e-9, 1e-10 ]
atol_values = [1e-6, 1e-6, 1e-9, 1e-10]
last_ = [rtol_values[-1], atol_values[-1]]
success=False
while success==False:
    atol = atol_values.pop(0)
    rtol = rtol_values.pop(0)
    try:
        result_solve_ivp = solve_ivp(
            dydt,
            t_span,
            initial_conc,
            method="BDF",
            dense_output=True,
            rtol=rtol,
            atol=atol,
            jac=dydt.jac,
            max_step=max_step,
            timeout=timeout,
        )
        #timeout
        if result_solve_ivp == "Shiki": 
            if rtol == last_[0] and atol == last_[1]:
                success=True
                cont=True
            continue
        else: success=True
    
    # should be arraybox failure
    except Exception as e:
        if rtol == last_[0] and atol == last_[1]:
            success=True
            cont=True
        continue

if cont:
    try:
        result_solve_ivp = solve_ivp(
            dydt,
            t_span,
            initial_conc,
            method="LSODA",
            dense_output=True,
            rtol=1e-6,
            atol=1e-9,
            max_step=max_step,
            timeout=timeout,
        )
    except Exception as e:
        # Last resort
        result_solve_ivp = solve_ivp(
            dydt,
            t_span,
            initial_conc,
            method="Radau",
            dense_output=True,
            rtol=1e-6,
            atol=1e-9,
            max_step=max_step,
            jac=dydt.jac,
            timeout=timeout,
        )
    
if result_solve_ivp.success:      
    idx_target_all = [states.index(i) for i in states if "*" in i]
    c_target_t = [result_solve_ivp.y[i][-1] for i in idx_target_all]
else: 
    

In [105]:
result_solve_ivp = solve_ivp(
    dydt,
    t_span,
    initial_conc,
    method="LSODA",
    dense_output=True,
    rtol=1e-6,
    atol=1e-9,
    # jac=dydt.jac,
    max_step=max_step,
    timeout=timeout,
)
result_solve_ivp.success

True

In [ ]:
from tqdm import tqdm
import time

prod_conc_pt = []
for profile in tqdm(d, total=len(d), ncols=80):
    
    try:
        result = calc_km([profile[:-1]], [profile[-1]], 298.15, coeff_TS_all, df_network, (0, 1e5), c0, timeout=120)
        if result is None:
            prod_conc_pt.append(np.nan)
            continue
        prod_conc_pt.append(result[0]) 
    except Exception as e:
        print(e)
        prod_conc_pt.append(np.nan)


In [ ]:
prod_conc_pt = np.array(prod_conc_pt)
descrp_pt = np.array([i[descp_idx] for i in d ])

#sort both 
sort_indices = np.argsort(descr_all)
descrp_pt = descrp_pt[sort_indices]
prod_conc_pt = prod_conc[sort_indices]

# to avoid interpolating beyond the range
if prod_conc_pt[0] == np.NaN:
    descrp_pt = descrp_pt[1:]
    prod_conc_pt = prod_conc_pt[1:]
elif prod_conc_pt[-1] == np.NaN:
    descrp_pt = descrp_pt[:-1]
    prod_conc_pt = prod_conc_pt[:-1] 

if np.any(np.isnan(prod_conc_pt)):
    missing_indices = np.isnan(prod_conc_pt)
    f = interp1d(descrp_pt[~missing_indices], prod_conc_pt[~missing_indices], kind='cubic')
    y_interp = f(descrp_pt[missing_indices])
    prod_conc_pt_ = prod_conc_pt.copy()
    prod_conc_pt_[missing_indices] = y_interp
else:
    prod_conc_pt_ = prod_conc_pt.copy()
    

In [ ]:
xlabel = f"{tag} [kcal/mol]"
ylabel = "Product concentraion (M)"

plot_2d(descr_all, prod_conc_, descrp_pt, prod_conc_pt_, \
        xmin = xmin, xmax = xmax, ybase = 0.1, cb=cb, ms=ms, \
                xlabel=xlabel, ylabel=ylabel, plotmode=3)

## dealing with rough plot

In [ ]:
np.savetxt("descr_all.txt", descr_all, )
np.savetxt( "prod_conc_.txt",prod_conc_,)